# Variational Quantum Deflation (VQD)

Variational Quantum Deflation (VQD) extends the Variational Quantum Eigensolver (VQE) so you can compute not only the ground state but also excited states. After finding the lowest eigenpair $(\lambda_0, \ket{\psi_0})$, VQD “deflates” the Hamiltonian by adding a penalty term:
\begin{equation*}
H^{(1)} = H \;+\; \beta_0\,\ket{\psi_0}\!\bra{\psi_0}
\end{equation*}
which raises the energy of $\ket{\psi_0}$ and forces the next variational optimization to converge to the first excited eigenpair $(\lambda_1, \ket{\psi_1})$. You repeat this deflation step $k$ times to obtain the lowest $k$ eigenvalues and eigenstates.

For a hands-on walkthrough and code examples, see the Qiskit course [Qiskit VQD](https://learning.quantum.ibm.com/course/variational-algorithm-design/instances-and-extensions#variational-quantum-deflation-vqd) and the original paper [Variational Quantum Computation of Excited States](http://dx.doi.org/10.22331/q-2019-07-01-156).


In [1]:
from qiskit import QuantumCircuit
import numpy as np
from scipy.optimize import minimize
from qiskit.primitives import StatevectorEstimator
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import SparsePauliOp, Statevector, Operator
from qiskit.circuit import Parameter

estimator = StatevectorEstimator()

### How we update the Hamiltonian  
At each deflation step $i$, we take the previously found states $\{\ket{\psi_0},\dots,\ket{\psi_{i-1}}\}$ and coefficients $\{\beta_0,\dots,\beta_{i-1}\}$, and form
\begin{equation*}
  H^{(i)} = H^{(0)} \;+\;\sum_{j=0}^{i-1}\beta_j\,\ket{\psi_j}\!\bra{\psi_j}.
\end{equation*}
This “pushes up” the energies of all earlier states, so the optimizer can land on the next excitation.


In [2]:
def build_vqd_observable(observable: SparsePauliOp, previous_states: list[Statevector], beta: list[float]) -> SparsePauliOp:

    if len(previous_states) != len(beta):
        raise ValueError("Length of beta must match number of previous states.")

    penalty = SparsePauliOp.from_list([("I" * observable.num_qubits, 0.0)])

    for psi, b in zip(previous_states, beta):
        projector = psi.to_operator()  # full matrix
        pauli_proj = SparsePauliOp.from_operator(projector)
        penalty += b * pauli_proj

    return observable + penalty


We can reuse the cost function and the optimization loop from VQE using the updated hamiltonian

In [3]:
def cost_func_vqe(parameters, ansatz, observable, estimator):
    estimator_job = estimator.run([(ansatz, observable, [parameters])])
    estimator_result = estimator_job.result()[0]

    cost = estimator_result.data.evs[0]
    return cost

def opti_loop_vqe(cost_func_vqe, x0, ansatz, observable, estimator, method="COBYLA"):
    return minimize(cost_func_vqe, x0, args=(ansatz, observable, estimator), method=method)


In [4]:
def run_vqd(ansatz, observable, k, estimator, beta_values,
            x0_list=None, method="COBYLA"):
    eigenvalues = []
    previous_states = []

    for i in range(k):
        # 1. Deflated observable
        updated_obs = build_vqd_observable(observable,
                                           previous_states,
                                           beta_values[:i])

        # 2. Pick initial guess:
        if x0_list is not None and len(x0_list) > i:
            x0 = x0_list[i]
        else:
            x0 = np.ones(ansatz.num_parameters)

        # 3. Optimize
        result = opti_loop_vqe(cost_func_vqe, x0,
                               ansatz, updated_obs,
                               estimator, method)

        # 4. Record
        eigenvalues.append(result.fun)
        previous_states.append(Statevector(ansatz.assign_parameters(result.x)))

    return eigenvalues, previous_states


### Test Case 1 — 2-Qubit Non-degenerate Hamiltonian

**Hamiltonian**  
\begin{equation*}
H = Z\otimes Z \;+\; 2\,X\otimes X
\end{equation*}

**Spectrum**  
-3, -1, +1, +3

**Eigenstates**  
- $\displaystyle \ket{\psi_0} = \frac{1}{\sqrt2}\bigl(\ket{01} - \ket{10}\bigr)$  
- $\displaystyle \ket{\psi_1} = \frac{1}{\sqrt2}\bigl(\ket{00} - \ket{11}\bigr)$  
- $\displaystyle \ket{\psi_2} = \frac{1}{\sqrt2}\bigl(\ket{01} + \ket{10}\bigr)$  
- $\displaystyle \ket{\psi_3} = \frac{1}{\sqrt2}\bigl(\ket{00} + \ket{11}\bigr)$



In [5]:
# 1. Build the 2-qubit ansatz
θ0, θ1 = Parameter('θ0'), Parameter('θ1')
ansatz1 = QuantumCircuit(2)
ansatz1.ry(θ0, 0)
ansatz1.ry(θ1, 1)
ansatz1.cx(0, 1)

# 2. Define the Hamiltonian H = Z⊗Z + 2 X⊗X
obs1 = SparsePauliOp.from_list([
    ("ZZ", 1.0),
    ("XX", 2.0)
])

# 3. Run VQD for the first four eigenvalues (k=4)
beta_vals = [10, 20, 40, 80]
eigenvals, states = run_vqd(
    ansatz1,
    obs1,
    k=4,
    estimator=estimator,
    beta_values=beta_vals,
    method="COBYLA"
)

# 4. Fix global phase so that the first amplitude is real & positive
def fix_global_phase(vec: np.ndarray, threshold: float = 1e-3) -> np.ndarray:
    """
    Rotate the global phase so that the first amplitude with |amp|>threshold
    becomes real and positive.
    """
    v = vec.copy()
    # find first significant index
    for amp in v:
        if abs(amp) > threshold:
            phi = np.angle(amp)
            v *= np.exp(-1j * phi)
            if v[np.argmax(np.abs(v))].real < 0:
                v *= -1
            break
    return v

# Sort eigenvalues with their states
pairs = sorted(zip(eigenvals, states), key=lambda x: x[0])

# Print each eigenvalue and its statevector on one line
threshold = 1e-3
for ev, st in pairs:
    vec = fix_global_phase(st.data)
    terms = [
        f"({amp:.5f})|{format(idx, '02b')}⟩"
        for idx, amp in enumerate(vec)
        if abs(amp) > threshold
    ]
    sv = " + ".join(terms)
    print(f"λ = {ev:.6f} → {sv}")


λ = -3.000000 → (-0.70708-0.00000j)|01⟩ + (0.70713+0.00000j)|10⟩
λ = -1.000000 → (-0.70708+0.00000j)|00⟩ + (0.70714-0.00000j)|11⟩
λ = 1.000000 → (0.70713+0.00000j)|01⟩ + (0.70708+0.00000j)|10⟩
λ = 3.000000 → (0.70712+0.00000j)|00⟩ + (0.70710+0.00000j)|11⟩


### Test Case 2 — 3-Qubit Mixed-Interaction Hamiltonian using the TwoLocal ansatz

**Hamiltonian**  
\begin{equation*}
H = X\otimes X\otimes Z \;+\; Y\otimes Y\otimes Z \;+\; Z\otimes Z\otimes Z
\end{equation*}

**Spectrum (degenerate)**
-3, -1, +1, +3


**Eigenstates (only the first 4)**
\begin{align*}
\lambda_0 &= -3, & \ket{\psi_0} &= \tfrac{1}{\sqrt2}\bigl(\ket{010} - \ket{100}\bigr),\\
\lambda_1 &= -1, & \ket{\psi_1} &= \ket{001},\\
\lambda_2 &= -1, & \ket{\psi_2} &= \tfrac{1}{\sqrt2}\bigl(\ket{011} + \ket{101}\bigr),\\
\lambda_3 &= -1, & \ket{\psi_3} &= \ket{111}.
\end{align*}



In [6]:
# 1. TwoLocal ansatz
ansatz2 = TwoLocal(num_qubits=3,
                   rotation_blocks=["rz","ry"],
                   entanglement_blocks="cx",
                   entanglement="linear",
                   reps=1)

# 2. Hamiltonian
obs2 = SparsePauliOp.from_list([
    ("XXZ", 1.0),
    ("YYZ", 1.0),
    ("ZZZ", 1.0)
])


# Generate exactly the same kind of x0 you used in VQE:
x0_2 = np.random.uniform(0, 2*np.pi, size=len(ansatz2.parameters))

# Build a list of guesses: reuse x0_2 for the first deflation,
# then maybe randomize for the rest
x0_list = [x0_2] + [
    np.random.uniform(0, 2*np.pi, size=len(ansatz2.parameters))
    for _ in range(3)
]

beta_vals = [5, 10, 20, 40,80]
eigenvals2, states2 = run_vqd(
    ansatz2, obs2, k=5, estimator=estimator,
    beta_values=beta_vals, x0_list=x0_list
)

# Sort eigenvalues with their states
pairs = sorted(zip(eigenvals2, states2), key=lambda x: x[0])

# Print each eigenvalue and its statevector on one line
threshold = 1e-3
for ev, st in pairs:
    vec = fix_global_phase(st.data, threshold)
    terms = [
        f"({amp:.5f})|{format(idx, '03b')}⟩"
        for idx, amp in enumerate(vec)
        if abs(amp) > threshold
    ]
    sv = " + ".join(terms)
    print(f"λ = {ev:.6f} → {sv}")


λ = -3.000000 → (-0.70708-0.00000j)|010⟩ + (0.70713-0.00001j)|100⟩
λ = -1.000000 → (-0.38625+0.00000j)|001⟩ + (-0.04691-0.50849j)|011⟩ + (-0.04675-0.50826j)|101⟩ + (0.53268+0.21399j)|111⟩
λ = -1.000000 → (0.52187-0.00000j)|001⟩ + (-0.15222-0.45490j)|011⟩ + (-0.15187-0.45475j)|101⟩ + (-0.51738+0.00234j)|111⟩
λ = -0.992706 → (0.00971-0.00000j)|000⟩ + (0.72662+0.21046j)|001⟩ + (-0.01379-0.00485j)|010⟩ + (0.06780+0.07043j)|011⟩ + (-0.00928-0.00334j)|100⟩ + (0.05984+0.09074j)|101⟩ + (-0.00938-0.00683j)|110⟩ + (0.58225+0.25847j)|111⟩
λ = 1.000502 → (0.30017+0.00000j)|000⟩ + (0.00967+0.00130j)|001⟩ + (0.59519+0.07778j)|010⟩ + (0.00612+0.00131j)|011⟩ + (0.59355+0.07853j)|100⟩ + (0.00088+0.00889j)|101⟩ + (0.13790+0.41449j)|110⟩ + (0.00971+0.00138j)|111⟩


In the presence of degeneracy (here, three eigenvalues equal to −1), VQD will return some basis of that degenerate subspace—but not necessarily the exact eigenvectors define before. To verify that our three VQD-computed “excited” states truly span the correct λ = −1 subspace, we use the **projector test**:

1. **Exact projector**  
   \begin{equation*}
     P_{\rm exact}
     = \sum_{j:\,\lambda_j=-1} \ket{\psi_j}\!\bra{\psi_j}
     = \Psi\,\Psi^\dagger
   \end{equation*}
   built from the three known eigenvectors at λ = −1.

2. **VQD projector**  
   \begin{equation*}
     P_{\rm VQD}
     = \sum_{j=1}^3 \ket{\phi_j}\!\bra{\phi_j}
     = \Phi\,\Phi^\dagger
   \end{equation*}
   built from the three VQD-computed states.

3. **Frobenius-norm check**  
   \begin{equation*}
     \delta = \|P_{\rm exact} - P_{\rm VQD}\|_F.
   \end{equation*}
   If $\delta \approx 0$ (up to numerical noise), the two projectors—and therefore the two 3-dimensional subspaces—are identical.  

This test is **basis-independent** and gives a single answer about whether VQD captured the correct subspace.  


In [7]:
# 1) Exact λ = −1 subspace (3 vectors in 8-dim Hilbert space)
#    Basis: 000=0,001=1,010=2,011=3,100=4,101=5,110=6,111=7
psi = np.zeros((8, 3), complex)
psi[1,0] = 1.0                                  # |001⟩
psi[3,1] = 1/np.sqrt(2); psi[5,1] = 1/np.sqrt(2) # (|011⟩+|101⟩)/√2
psi[7,2] = 1.0                                  # |111⟩

# 2) Extract your VQD-computed states at λ≈−1
phi = np.column_stack([states2[i].data for i in [1, 2, 3]])  # shape (8,3)
#phi = np.column_stack([fix_global_phase(col) for col in phi.T])

# 3) Build projectors
P_exact = psi @ psi.conj().T
P_vqd   = phi @ phi.conj().T

# 4) Frobenius-norm of their difference
delta = np.linalg.norm(P_exact - P_vqd, ord='fro')
print(f"Projector difference ‖P_exact – P_vqd‖ₚ = {delta:.6f}")


Projector difference ‖P_exact – P_vqd‖ₚ = 0.048467


### Test case 3 - Random 3-Qubit Hamiltonian

1. **Generate** a random 3-qubit Hermitian Hamiltonian $H$.  
2. **Diagonally** solve it exactly (ground + first three excited states) using SciPy.  
3. **Variationally** approximate the same four eigenpairs with VQD.  
4. **Compare** the classical and variational results side by side.


In [8]:
def random_hermitian_matrix(n):
    dim = 2**n
    A = 2*(np.random.rand(dim, dim) + 1j*np.random.rand(dim, dim))
    H = (A + A.conj().T) / 2  # Make it Hermitian
    return H

n=3
H_matrix = random_hermitian_matrix(n)

We use `scipy.linalg.eigh` to obtain all eigenvalues/vectors, then display the lowest four.

In [9]:
from scipy.linalg import eigh

# Diagonalize
eigenvals, eigenvecs = eigh(H_matrix)

# Take the first four eigenpairs
first4_vals = eigenvals[:4]
first4_vecs = eigenvecs[:, :4]  

# Print each eigenvalue and its statevector
threshold = 1e-3
for val, vec in zip(first4_vals, first4_vecs.T):
    vec_fixed = fix_global_phase(vec, threshold)
    terms = [
        f"({amp:.5f})|{format(idx, f'0{int(np.log2(len(vec_fixed)))}b')}⟩"
        for idx, amp in enumerate(vec_fixed)
        if abs(amp) > threshold
    ]
    print(f"λ = {val:.6f} → " + " + ".join(terms))


λ = -2.182468 → (0.33097+0.00000j)|000⟩ + (-0.08077+0.27563j)|001⟩ + (-0.35390+0.43137j)|010⟩ + (0.11342-0.25797j)|011⟩ + (-0.12076+0.00492j)|100⟩ + (0.10701-0.55262j)|101⟩ + (0.14955+0.12691j)|110⟩ + (-0.21077-0.05373j)|111⟩
λ = -1.178976 → (-0.07904+0.00000j)|000⟩ + (-0.27534-0.22125j)|001⟩ + (-0.08499-0.05336j)|010⟩ + (0.46780-0.37390j)|011⟩ + (0.21217+0.56351j)|100⟩ + (-0.17193-0.01077j)|101⟩ + (0.00843-0.07380j)|110⟩ + (0.15799+0.27852j)|111⟩
λ = -0.550601 → (-0.30902+0.00000j)|000⟩ + (-0.09001+0.18983j)|001⟩ + (0.03426-0.36230j)|010⟩ + (0.06081+0.20845j)|011⟩ + (0.44200+0.01138j)|100⟩ + (-0.17811-0.29764j)|101⟩ + (0.42327+0.09216j)|110⟩ + (-0.39174-0.15451j)|111⟩
λ = 0.018098 → (-0.39864+0.00000j)|000⟩ + (0.61895+0.08380j)|001⟩ + (-0.28820-0.01234j)|010⟩ + (0.14227-0.26321j)|011⟩ + (-0.07756+0.11558j)|100⟩ + (-0.16155+0.05516j)|101⟩ + (-0.14377+0.27448j)|110⟩ + (0.07856-0.35713j)|111⟩


Variational Quantum Deflation (VQD) Approximation :


In [10]:
# 1. Build the 4-qubit TwoLocal ansatz
ansatz3 = TwoLocal(
    num_qubits=n,
    rotation_blocks=["rz", "ry"],
    entanglement_blocks="cx",
    entanglement="linear",
    reps=2
)

# 2. Hamiltonian from random H_matrix
obs3 = SparsePauliOp.from_operator(Operator(H_matrix))

# 3. Run VQD for the first four eigenvalues (k=4)
beta_vals = [5, 10, 20, 40]
eigenvals3, states3 = run_vqd(
    ansatz3,
    obs3,
    k=4,
    estimator=estimator,
    beta_values=beta_vals,
    method="COBYLA"
)

# 4. Sort eigenvalues and corresponding states
pairs3 = sorted(zip(eigenvals3, states3), key=lambda x: x[0])

# 5. Define phase-fixing helper
def fix_global_phase(vec: np.ndarray, threshold: float = 1e-3) -> np.ndarray:
    v = vec.copy()
    for amp in v:
        if abs(amp) > threshold:
            phi = np.angle(amp)
            v *= np.exp(-1j * phi)
            if v[np.where(abs(v)>threshold)[0][0]].real < 0:
                v *= -1
            break
    return v

# 6. Print each eigenvalue and its significant components
threshold = 1e-3
for ev, st in pairs3:
    vec = fix_global_phase(st.data, threshold)
    terms = [
        f"({amp:.5f})|{format(idx, f'0{int(np.log2(len(vec_fixed)))}b')}⟩"
        for idx, amp in enumerate(vec)
        if abs(amp) > threshold
    ]
    print(f"λ = {ev:.6f} →", " + ".join(terms))

λ = -2.160163 → (0.33284-0.00000j)|000⟩ + (-0.05474+0.24926j)|001⟩ + (-0.34265+0.41266j)|010⟩ + (0.18122-0.24775j)|011⟩ + (-0.18030+0.04919j)|100⟩ + (0.12531-0.55679j)|101⟩ + (0.10790+0.14115j)|110⟩ + (-0.22188-0.02706j)|111⟩
λ = -1.146385 → (0.11511+0.00000j)|000⟩ + (0.04348+0.31605j)|001⟩ + (0.09118+0.17186j)|010⟩ + (-0.61601+0.12103j)|011⟩ + (0.13062-0.55088j)|100⟩ + (0.13294-0.00634j)|101⟩ + (0.08525+0.03287j)|110⟩ + (-0.10898-0.30748j)|111⟩
λ = -0.563608 → (0.28631-0.00000j)|000⟩ + (0.16374-0.15107j)|001⟩ + (-0.00925+0.35281j)|010⟩ + (-0.13105-0.17743j)|011⟩ + (-0.44595-0.09515j)|100⟩ + (0.16438+0.32852j)|101⟩ + (-0.43103-0.09663j)|110⟩ + (0.37820+0.11894j)|111⟩
λ = 0.208027 → (0.52332+0.00000j)|000⟩ + (-0.61096-0.09010j)|001⟩ + (0.12312-0.07866j)|010⟩ + (-0.17684+0.16934j)|011⟩ + (-0.01224+0.10109j)|100⟩ + (0.06279-0.07037j)|101⟩ + (0.11826-0.34385j)|110⟩ + (-0.06030+0.32916j)|111⟩


Finally, we print the first four eigenpairs side-by-side, with the 2-norm 
between each exact and variational state to quantify the error.

In [11]:
print("Comparison of first 4 eigenpairs (Classical vs VQD):\n")
for i in range(4):
    # Classical
    cl_val = first4_vals[i]
    cl_vec = fix_global_phase(first4_vecs[:, i])
    # VQD (sorted results in pairs3)
    vq_val, vq_state = pairs3[i]
    vq_vec = fix_global_phase(vq_state.data)
    
    # Norm difference
    diff = np.linalg.norm(cl_vec - vq_vec)
    
    print(f"Eigenpair {i}:")
    print(f"  Classical λ = {cl_val:.6f}")
    print(f"  VQD       λ = {vq_val:.6f}")
    print(f"  ‖φ_cl – φ_vqd‖₂ = {diff:.6f}\n")

Comparison of first 4 eigenpairs (Classical vs VQD):

Eigenpair 0:
  Classical λ = -2.182468
  VQD       λ = -2.160163
  ‖φ_cl – φ_vqd‖₂ = 0.123226

Eigenpair 1:
  Classical λ = -1.178976
  VQD       λ = -1.146385
  ‖φ_cl – φ_vqd‖₂ = 0.545162

Eigenpair 2:
  Classical λ = -0.550601
  VQD       λ = -0.563608
  ‖φ_cl – φ_vqd‖₂ = 0.154170

Eigenpair 3:
  Classical λ = 0.018098
  VQD       λ = 0.208027
  ‖φ_cl – φ_vqd‖₂ = 0.364364

